# Anlisis detallado escenario base

In [11]:
# %% Import Libraries
import json
import matplotlib.pyplot as plt # Still needed for TMT summary plot if run for all
import numpy as np
import pandas as pd
from collections import defaultdict
import os
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import traceback # For detailed error reporting

# %% Configuration
# Set default Plotly template
# --- CORRECTED TEMPLATE NAME ---
pio.templates.default = "plotly_white" # Standard template for white background
# -------------------------------

json_file_path = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_second_optimization.json"
# Output directory for results - will be created if it doesn't exist
output_dir = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/second_coordination_plots"

# Coordination Time Interval (CTI)
CTI = 0.2

# %% Helper Functions

def calculate_MT(tm, tb, cti=CTI):
    """Calculates the Miscoordination Index (MT) and the time difference delta."""
    delta_tmb = tb - tm - cti
    MT = (delta_tmb - abs(delta_tmb)) / 2
    return MT, delta_tmb

def analyze_by_scenario(data, cti=CTI):
    """
    Analyzes relay pair data, calculating MT, Delta_t, TMT,
    and extracting additional info (line, fault) for each scenario.
    Returns a dictionary mapping scenario_id to its analysis results (DataFrame, TMT).
    """
    scenarios = defaultdict(lambda: {"pairs_data": []})
    print("Starting data extraction and processing for scenarios...")
    processed_count = 0
    skipped_count = 0
    # --- Data Extraction and Initial Processing ---
    for idx, relay_pair in enumerate(data):
        record_label = f"Record {idx+1}"
        scenario_id = relay_pair.get("scenario_id")
        if not scenario_id:
            # print(f"Warning: Skipping record {idx+1} - Missing 'scenario_id'.")
            skipped_count += 1
            continue

        record_label = f"{record_label} (Scenario: {scenario_id})"
        main_relay = relay_pair.get("main_relay")
        backup_relay = relay_pair.get("backup_relay")
        if not main_relay or not backup_relay:
            # print(f"Warning: Skipping {record_label} - Missing 'main_relay' or 'backup_relay' data.")
            skipped_count += 1
            continue

        tm_val = main_relay.get("Time_out")
        tb_val = backup_relay.get("Time_out")
        main_relay_id = main_relay.get("relay", f"Main_Unknown_{idx+1}")
        backup_relay_id = backup_relay.get("relay", f"Backup_Unknown_{idx+1}")
        pair_name = f"{main_relay_id}-{backup_relay_id}"
        record_label = f"{record_label}, Pair: {pair_name}"

        if tm_val is None or tb_val is None:
            # print(f"Warning: Skipping {record_label} - Missing 'Time_out' for main or backup relay.")
            skipped_count += 1
            continue

        line_info = relay_pair.get('line', main_relay.get('line', backup_relay.get('line', 'N/A')))
        fault_info = relay_pair.get('fault', main_relay.get('fault', backup_relay.get('fault', 'N/A')))

        try:
             tm_float = float(tm_val)
             tb_float = float(tb_val)
             if tm_float < 0 or tb_float < 0:
                 # print(f"Warning: Skipping {record_label} - Negative time values detected (Tm={tm_float}, Tb={tb_float}).")
                 skipped_count += 1
                 continue
             mt, delta_t = calculate_MT(tm_float, tb_float, cti)
        except (ValueError, TypeError) as e:
             print(f"Warning: Could not calculate MT for {record_label}. Invalid time values? Tm='{tm_val}', Tb='{tb_val}'. Error: {e}")
             skipped_count += 1
             continue

        pair_info = {
            "pair_label": pair_name, "mt": mt, "delta_t": delta_t,
            "tm": tm_float, "tb": tb_float, "main_relay_id": main_relay_id,
            "backup_relay_id": backup_relay_id, "line": line_info,
            "fault": fault_info, "status": "Coordinado" if delta_t >= 0 else "Descoordinado"
        }
        scenarios[scenario_id]["pairs_data"].append(pair_info)
        processed_count += 1

    print(f"Data extraction complete. Processed pairs: {processed_count}, Skipped records/pairs: {skipped_count}")

    # --- Aggregation per Scenario ---
    print("Aggregating results per scenario...")
    results = {}
    all_scenario_ids = sorted(list(scenarios.keys()))

    if not all_scenario_ids:
        print("No scenarios found with valid data after processing.")
        return results

    for scenario_id in all_scenario_ids:
        scenario_pairs_list = scenarios[scenario_id]["pairs_data"]
        if not scenario_pairs_list:
            # print(f"Info: Scenario '{scenario_id}' has no valid pairs after processing.")
            continue
        try:
            df = pd.DataFrame(scenario_pairs_list)
            tmt = df["mt"].sum()
            results[scenario_id] = {"detailed_df": df, "TMT": tmt}
            # print(f"Scenario '{scenario_id}' processed. TMT: {tmt:.4f}, Pairs: {len(df)}")
        except Exception as e:
            print(f"Error creating DataFrame or calculating TMT for scenario '{scenario_id}': {e}")
            continue

    print("Scenario aggregation complete.")
    return results


# %% Plotting Functions

def plot_scenario_details_plotly(scenario_data_df, scenario_id, output_dir, cti=CTI):
    """
    Generates, SAVES, and DISPLAYS multiple interactive Plotly charts for a scenario:
    1. MT Bar Chart
    2. Delta_t Bar Chart  <-- NEW
    3. Delta_t Scatter Plot
    4. MT Histogram
    """
    if not isinstance(scenario_data_df, pd.DataFrame) or scenario_data_df.empty:
        print(f"Plotting skipped for scenario {scenario_id}: No valid DataFrame provided.")
        return

    print(f"\n--- Generating Plots for Scenario: {scenario_id} ---")
    # Sort by MT for consistent ordering in MT and Delta_t plots
    df_sorted_mt = scenario_data_df.sort_values(by='mt', ascending=True).reset_index(drop=True)
    df_sorted_mt['plot_index'] = df_sorted_mt.index + 1
    tmt = df_sorted_mt["mt"].sum()
    n_pairs = len(df_sorted_mt)

    # Common hover text generation logic (avoids repetition)
    def generate_hover_text(row):
        return (f"<b>{row['pair_label']}</b><br>Índice Plot: {row['plot_index']}<br>" +
                f"MT: {row['mt']:.4f} | <b>Δt: {row['delta_t']:.4f}s</b><br>" +
                f"Tm: {row['tm']:.4f}s | Tb: {row['tb']:.4f}s<br>Línea: {row['line']} | Falla: {row['fault']}<br>" +
                f"Estado: {row['status']}<extra></extra>")

    hover_texts = [generate_hover_text(row) for _, row in df_sorted_mt.iterrows()]

    # --- 1. MT Bar Chart (Shows all pairs, positive and negative MT) ---
    try:
        print(f"  Generating MT Bar Chart...")
        colors_mt = ['red' if mt < 0 else 'green' for mt in df_sorted_mt['mt']]
        fig_mt_bar = go.Figure(data=[go.Bar(
            x=df_sorted_mt['plot_index'], y=df_sorted_mt['mt'], marker_color=colors_mt,
            name='Índice MT', text=hover_texts, hoverinfo='text'
        )])
        fig_mt_bar.add_hline(y=0, line_width=1.5, line_dash="dash", line_color="black")
        fig_mt_bar.update_layout(
            title=f'Índice de Descoordinación (MT) - Escenario: {scenario_id} Optimizado<br><sup>TMT = {tmt:.4f} | Pares = {n_pairs}</sup>',
            xaxis_title='Índice del Par de Relés (Ordenado por MT Ascendente)', yaxis_title='Índice MT',
            xaxis=dict(tickmode='auto'),
            yaxis=dict(gridcolor='lightgrey', zerolinecolor='black', zerolinewidth=1.5),
            bargap=0.1, height=max(500, 400 + n_pairs * 5), margin=dict(l=60, r=30, t=100, b=100)
        )
    except Exception as e:
        print(f"    Error generating MT Bar Chart: {e}")
        fig_mt_bar = None

    # --- 2. Delta_t Bar Chart (Shows all pairs, positive and negative Delta_t) --- <<< NEW PLOT
    try:
        print(f"  Generating Delta_t Bar Chart...")
        colors_delta = ['red' if delta_t < 0 else 'green' for delta_t in df_sorted_mt['delta_t']]
        fig_delta_bar = go.Figure(data=[go.Bar(
            x=df_sorted_mt['plot_index'], y=df_sorted_mt['delta_t'], marker_color=colors_delta,
            name='Δt (Tb - Tm - CTI)', text=hover_texts, hoverinfo='text'
        )])
        fig_delta_bar.add_hline(y=0, line_width=1.5, line_dash="dash", line_color="black",
                                annotation_text="Límite CTI (Δt=0)", annotation_position="bottom right")
        fig_delta_bar.update_layout(
            title=f'Diferencia de Tiempo (Δt = Tb - Tm - CTI) - Escenario: {scenario_id} Optimizado<br><sup>CTI = {cti}s</sup>',
            xaxis_title='Índice del Par de Relés (Ordenado por MT Ascendente)', yaxis_title='Δt [s]',
            xaxis=dict(tickmode='auto'),
            yaxis=dict(gridcolor='lightgrey', zerolinecolor='black', zerolinewidth=1.5),
            bargap=0.1, height=max(500, 400 + n_pairs * 5), margin=dict(l=60, r=30, t=100, b=100)
        )
    except Exception as e:
        print(f"    Error generating Delta_t Bar Chart: {e}")
        fig_delta_bar = None

    # --- 3. Delta_t Scatter Plot (Shows all pairs, colors by status) ---
    try:
        print(f"  Generating Delta_t Scatter Plot...")
        fig_delta_scatter = px.scatter(
            df_sorted_mt, x='plot_index', y='delta_t', color='status',
            color_discrete_map={'Coordinado': 'green', 'Descoordinado': 'red'},
            hover_data={'plot_index': False, 'pair_label': True, 'delta_t': ':.4f', 'mt': ':.4f',
                        'tm': ':.4f', 'tb': ':.4f', 'line': True, 'fault': True, 'status': True},
            title=f'Diferencia de Tiempo (Δt vs Índice) - Escenario: {scenario_id} Optimizado<br><sup>CTI = {cti}s</sup>'
        )
        fig_delta_scatter.add_hline(y=0, line_width=1.5, line_dash="dash", line_color="black",
                                     annotation_text="Límite CTI (Δt=0)", annotation_position="bottom right")
        fig_delta_scatter.update_layout(
            xaxis_title='Índice del Par de Relés (Ordenado por MT Ascendente)', yaxis_title='Δt (Tb - Tm - CTI) [s]',
            xaxis=dict(tickmode='auto'), yaxis=dict(gridcolor='lightgrey', zerolinecolor='black', zerolinewidth=1.5),
            legend_title_text='Estado Coordinación'
        )
        fig_delta_scatter.update_traces(marker=dict(size=8, opacity=0.7), selector=dict(mode='markers'))
    except Exception as e:
        print(f"    Error generating Delta_t Scatter Plot: {e}")
        fig_delta_scatter = None

    # --- 4. MT Histogram ---
    try:
        print(f"  Generating MT Histogram...")
        fig_mt_hist = px.histogram(
            df_sorted_mt, x='mt', color='status',
            color_discrete_map={'Coordinado': 'green', 'Descoordinado': 'red'},
            marginal='box', nbins=max(10, n_pairs // 5),
            title=f"Distribución de Valores MT - Escenario: {scenario_id} Optimizado"
        )
        fig_mt_hist.update_layout(
            xaxis_title='Valor MT', yaxis_title='Frecuencia (Número de Pares)',
            legend_title_text='Estado Coordinación', bargap=0.1
        )
    except Exception as e:
        print(f"    Error generating MT Histogram: {e}")
        fig_mt_hist = None

    # --- Save Plots ---
    plots_to_save = {
        "mt_bar": fig_mt_bar,
        "delta_bar": fig_delta_bar, # <<< Added new plot
        "delta_scatter": fig_delta_scatter,
        "mt_histogram": fig_mt_hist
    }
    print(f"  Saving plots for {scenario_id}...")
    kaleido_error_shown = False
    for name, fig in plots_to_save.items():
        if fig is None:
            print(f"    Skipping saving/display for '{name}' plot (generation failed).")
            continue
        base_filename = os.path.join(output_dir, f"{scenario_id}_{name}") # Use os.path.join
        html_filename = f"{base_filename}.html"
        png_filename = f"{base_filename}.png"

        # Save HTML
        try: fig.write_html(html_filename)
        except Exception as e: print(f"    Error saving HTML plot '{html_filename}': {e}")

        # Save PNG (optional, requires kaleido)
        try: fig.write_image(png_filename, scale=2)
        except ValueError as e:
            if "kaleido" in str(e) and not kaleido_error_shown:
                print("\n    Info: Install kaleido for static PNG export: pip install -U kaleido")
                print("    Skipping further PNG saving attempts if Kaleido is missing.\n")
                kaleido_error_shown = True
            elif "kaleido" not in str(e): print(f"    Error saving static image '{png_filename}': {e}")
        except Exception as e: print(f"    Unexpected error saving static image '{png_filename}': {e}")

        # --- DISPLAY PLOT IN JUPYTER/OUTPUT ---
        print(f"    Displaying {name} plot...")
        fig.show() # This line displays the plot in the notebook output or default browser
        # ---------------------------------

    print(f"--- Plot generation, saving, and display complete for Scenario: {scenario_id} ---")


# %% Main Execution Logic - Focused on Single Scenario

def main_analyze_single_scenario(data, scenario_id_to_analyze, cti=CTI):
    """
    Loads data, performs analysis, generates detailed table, stats, and plots
    SPECIFICALLY for the given scenario_id_to_analyze.
    """
    print(f"--- Starting Analysis for Single Scenario: {scenario_id_to_analyze} ---")
    print(f"Using CTI = {cti}s")

    # --- Ensure Output Directory Exists ---
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"Results will be saved in: {output_dir}")
    except OSError as e:
        print(f"Error: Could not create output directory '{output_dir}'. Check permissions. Error: {e}")
        return

    # --- Analyze Data (for all scenarios, then filter) ---
    all_results = analyze_by_scenario(data, cti)

    if not all_results:
        print("Error: Analysis yielded no results. Please check input data and analysis function.")
        return

    # --- Detailed Analysis for the Specified Scenario ---
    if scenario_id_to_analyze in all_results:
        print(f"\n--- Processing Detailed Analysis for: {scenario_id_to_analyze} ---")
        scenario_result = all_results[scenario_id_to_analyze]
        detailed_df = scenario_result.get("detailed_df")

        if detailed_df is not None and not detailed_df.empty:
            try:
                detailed_df_sorted = detailed_df.sort_values(by="mt", ascending=True).reset_index(drop=True)
            except KeyError:
                 print(f"Error: 'mt' column not found in DataFrame for scenario {scenario_id_to_analyze}. Cannot proceed.")
                 return

            # --- Print Coordination Summary Statistics ---
            print("\n--- Resumen de Coordinación ---") # <<< Renamed section for clarity
            n_total = len(detailed_df_sorted)
            n_coord = len(detailed_df_sorted[detailed_df_sorted['status'] == 'Coordinado'])
            n_descoord = n_total - n_coord
            coord_pct = (n_coord / n_total * 100) if n_total > 0 else 0
            descoord_pct = (n_descoord / n_total * 100) if n_total > 0 else 0
            print(f"Total Pares Analizados: {n_total}")
            print(f"Pares Coordinados (Δt >= 0): {n_coord} ({coord_pct:.1f}%)") # <<< Shows count and percentage
            print(f"Pares Descoordinados (Δt < 0): {n_descoord} ({descoord_pct:.1f}%)") # <<< Shows count and percentage

            print("\n--- Estadísticas del Índice MT ---")
            try:
                print(detailed_df_sorted['mt'].describe().to_string())
            except Exception as e:
                print(f"  Could not generate MT statistics: {e}")
            print(f"TMT (Suma MT): {scenario_result.get('TMT', 0):.5f}")

            # --- Identify and Print Worst Coordinated Pair --- <<< MODIFIED SECTION
            uncoordinated_pairs = detailed_df_sorted[detailed_df_sorted['status'] == 'Descoordinado']
            if not uncoordinated_pairs.empty:
                # Find the single worst pair (minimum MT)
                worst_pair = uncoordinated_pairs.loc[uncoordinated_pairs['mt'].idxmin()]

                print("\n--- Par Más Descoordinado (Peor MT) ---") # <<< Specific heading
                print(f"Par (P-R): {worst_pair['pair_label']}")
                print(f"MT:        {worst_pair['mt']:.4f}")
                print(f"Δt:        {worst_pair['delta_t']:.4f} s")
                print(f"Tm:        {worst_pair['tm']:.4f} s")
                print(f"Tb:        {worst_pair['tb']:.4f} s")
                print(f"Línea:     {worst_pair['line']}")
                print(f"Falla:     {worst_pair['fault']}")

                # --- Optionally print Top 10 list as before ---
                print("\n--- Top 10 Pares con Mayor Descoordinación (Ordenados por Peor MT) ---")
                worst_pairs_display = uncoordinated_pairs.head(10)
                display_cols = ['pair_label', 'mt', 'delta_t', 'tm', 'tb', 'line', 'fault']
                existing_display_cols = [col for col in display_cols if col in worst_pairs_display.columns]
                # Create a copy to avoid SettingWithCopyWarning
                worst_pairs_display_formatted = worst_pairs_display[existing_display_cols].copy()
                worst_pairs_display_formatted = worst_pairs_display_formatted.rename(
                    columns={'pair_label': 'Par (P-R)', 'delta_t': 'Δt', 'line': 'Línea', 'fault': 'Falla'})
                float_cols = worst_pairs_display_formatted.select_dtypes(include=['float']).columns
                for col in float_cols:
                     worst_pairs_display_formatted[col] = worst_pairs_display_formatted[col].map('{:.4f}'.format)
                pd.set_option('display.max_rows', 20)
                pd.set_option('display.width', 120)
                print(worst_pairs_display_formatted.to_string(index=False))
            else:
                print("\n--- ¡Excelente! No se encontraron pares descoordinados en este escenario. ---")

            # --- Save Detailed DataFrame to CSV ---
            csv_filename = os.path.join(output_dir, f"{scenario_id_to_analyze}_detailed_results.csv") # Use os.path.join
            print(f"\nSaving detailed results to CSV: {csv_filename}")
            try:
                detailed_df_sorted.to_csv(csv_filename, index=False, encoding='utf-8-sig')
                print(f"  Successfully saved.")
            except Exception as e:
                print(f"  Error saving detailed CSV: {e}")

            # --- Generate AND DISPLAY Plotly Plots ---
            plot_scenario_details_plotly(detailed_df_sorted, scenario_id_to_analyze, output_dir, cti)

        else:
            print(f"No valid pair data found in DataFrame to generate table or plots for {scenario_id_to_analyze}.")

    else:
        print(f"Error: El escenario '{scenario_id_to_analyze}' especificado no se encontró en los resultados del análisis.")
        available_scenarios = list(all_results.keys()) if all_results else "None"
        print(f"Available scenarios found: {available_scenarios}")

    print(f"\n--- Analysis Complete for Scenario: {scenario_id_to_analyze} ---")


# %% Script Entry Point - Execute Analysis for Scenario 1

if __name__ == "__main__":
    # This block executes when the script is run directly (or a cell in Jupyter)
    print("============================================")
    print("=== Relay Coordination Analysis Script ===")
    print("===       Mode: Analyze Scenario 1       ===")
    print("============================================")

    # --- Create dummy data and directory for execution if file doesn't exist ---
    # This is just for demonstration if the actual file isn't available in the environment
    # You should REPLACE json_file_path and output_dir with your actual paths above.
    if not os.path.exists(os.path.dirname(json_file_path)):
         os.makedirs(os.path.dirname(json_file_path), exist_ok=True)
         print(f"Created dummy directory: {os.path.dirname(json_file_path)}")

    if not os.path.exists(json_file_path):
        print(f"Warning: JSON file '{json_file_path}' not found. Creating a dummy file for demonstration.")
        dummy_data = [
            {"scenario_id": "scenario_1", "main_relay": {"relay": "P1", "Time_out": 0.1, "line": "L1", "fault": "F1"}, "backup_relay": {"relay": "R1", "Time_out": 0.35}}, # Coordinated dt=0.05
            {"scenario_id": "scenario_1", "main_relay": {"relay": "P2", "Time_out": 0.2, "line": "L2", "fault": "F1"}, "backup_relay": {"relay": "R2", "Time_out": 0.3}}, # Uncoordinated dt=-0.1, mt=-0.1
            {"scenario_id": "scenario_1", "main_relay": {"relay": "P3", "Time_out": 0.15, "line": "L1", "fault": "F2"}, "backup_relay": {"relay": "R3", "Time_out": 0.25}}, # Uncoordinated dt=-0.1, mt=-0.1
            {"scenario_id": "scenario_1", "main_relay": {"relay": "P4", "Time_out": 0.25, "line": "L3", "fault": "F3"}, "backup_relay": {"relay": "R4", "Time_out": 0.3}}, # Uncoordinated dt=-0.15, mt=-0.15 <- Worst
            {"scenario_id": "scenario_1", "main_relay": {"relay": "P5", "Time_out": 0.3, "line": "L2", "fault": "F2"}, "backup_relay": {"relay": "R5", "Time_out": 0.6}}, # Coordinated dt=0.1
            {"scenario_id": "scenario_2", "main_relay": {"relay": "P6", "Time_out": 0.1, "line": "L4", "fault": "F4"}, "backup_relay": {"relay": "R6", "Time_out": 0.4}}  # Different scenario
        ]
        try:
            with open(json_file_path, 'w', encoding='utf-8') as f:
                json.dump(dummy_data, f, indent=2)
            print(f"Successfully created dummy file: {json_file_path}")
        except Exception as e:
            print(f"Error creating dummy file: {e}")
            # Exit if dummy file creation fails and original wasn't found
            exit() # Or handle differently

    # --- Main execution ---
    try:
        print(f"Attempting to load data from: {json_file_path}")
        if not os.path.exists(json_file_path):
             abs_path = os.path.abspath(json_file_path)
             raise FileNotFoundError(f"JSON file not found: '{json_file_path}' (Absolute: '{abs_path}')")
        if not os.path.isfile(json_file_path):
             raise FileNotFoundError(f"Specified path is not a file: '{json_file_path}'")

        with open(json_file_path, 'r', encoding='utf-8') as file:
            first_char = file.read(1)
            if not first_char: raise ValueError("JSON file appears to be empty.")
            file.seek(0) # Rewind after checking
            data = json.load(file)

        if not isinstance(data, list):
            raise TypeError(f"Expected JSON root to be a List '[]', found type '{type(data)}'. Check JSON structure.")
        if not data:
             print("Warning: JSON file loaded, but the list is empty. No data to analyze.")
             # Decide if exiting is appropriate
             # exit() # Exit if no data - or remove this to let it proceed and show no results

        print(f"Successfully loaded {len(data)} records from JSON.")

        # --- Run the main analysis function, focusing ONLY on scenario_1 ---
        TARGET_SCENARIO = "scenario_3"
        # Check if data was loaded before proceeding
        if data:
            main_analyze_single_scenario(data, scenario_id_to_analyze=TARGET_SCENARIO, cti=CTI)
        else:
            print("Skipping analysis as no data was loaded.")


    # --- Exception Handling ---
    except FileNotFoundError as e: print(f"\n--- Error ---\nFile Not Found: {e}\n*** Please ensure the 'json_file_path' variable is set correctly at the top of the script. ***")
    except ValueError as e: print(f"\n--- Error ---\nValue Error: {e}") # Catch empty file error
    except TypeError as e: print(f"\n--- Error ---\nType Error: {e}") # Catch wrong JSON root type
    except json.JSONDecodeError as e: print(f"\n--- Error ---\nJSON Decode Error: {e.msg} at Line {e.lineno}, Col {e.colno}. Check JSON syntax.")
    except KeyError as e: print(f"\n--- Error ---\nMissing Key Error: Required key {e} not found in the JSON data. Check data structure consistency.")
    except Exception as e:
        print(f"\n--- An Unexpected Error Occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {str(e)}")
        print("\n--- Traceback ---")
        traceback.print_exc()
        print("-----------------")

    print("\n============================================")
    print("=== Script Execution Finished ===")
    print("============================================")

=== Relay Coordination Analysis Script ===
===       Mode: Analyze Scenario 1       ===
Attempting to load data from: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_second_optimization.json
Successfully loaded 6800 records from JSON.
--- Starting Analysis for Single Scenario: scenario_3 ---
Using CTI = 0.2s
Results will be saved in: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/second_coordination_plots
Starting data extraction and processing for scenarios...
Data extraction complete. Processed pairs: 6800, Skipped records/pairs: 0
Aggregating results per scenario...
Scenario aggregation complete.

--- Processing Detailed Analysis for: scenario_3 ---

--- Resumen de Coordinación ---
Total Pares Analizados: 100
Pares Coordinados (Δt >= 0): 75 (75.0%)
Pares Descoordinados (Δt < 0): 25 (25.0%)

--- Estadísticas del Índice MT ---
count    100.000000
mean      -0.190496
std        0.592184
min       -2.537300


    Displaying delta_bar plot...


    Displaying delta_scatter plot...


    Displaying mt_histogram plot...


--- Plot generation, saving, and display complete for Scenario: scenario_3 ---

--- Analysis Complete for Scenario: scenario_3 ---

=== Script Execution Finished ===


# 🚀🚀🚀

# Analisys

In [14]:
import json
import os
import copy
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# --- Constantes ---
CTI = 0.2  # Intervalo de tiempo de coordinación típico (en segundos)

# --- Ruta del archivo ---
input_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_second_optimization.json"

def analyze_all_scenarios():
    try:
        # 1. Cargar el archivo JSON
        print(f"Cargando datos desde: {input_file}")
        with open(input_file, 'r') as f:
            relay_pairs_data = json.load(f)
        
        if not isinstance(relay_pairs_data, list):
            raise TypeError(f"Error: El archivo {input_file} no contiene una lista JSON.")
        
        # Diccionarios para almacenar datos por escenario
        scenario_results = {}  # Estructura: {scenario_id: {'tmt': valor, 'coordinated': cantidad, 'uncoordinated': cantidad}}
        
        # 2. Procesar cada par y agrupar por escenario
        print("Calculando TMT por escenario...")
        total_pairs_read = 0
        skipped_pairs_count = 0
        
        for pair_entry in relay_pairs_data:
            total_pairs_read += 1
            if not isinstance(pair_entry, dict):
                skipped_pairs_count += 1
                continue
            
            scenario_id = pair_entry.get("scenario_id")
            if not scenario_id:
                skipped_pairs_count += 1
                continue
                
            # Inicializar datos del escenario si es la primera vez
            if scenario_id not in scenario_results:
                scenario_results[scenario_id] = {
                    'tmt': 0, 
                    'coordinated': 0, 
                    'uncoordinated': 0,
                    'total_valid': 0
                }
            
            # Obtener información de los relés
            main_relay_info = pair_entry.get('main_relay')
            backup_relay_info = pair_entry.get('backup_relay')
            
            if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
                skipped_pairs_count += 1
                continue
                
            # Obtener tiempos de operación
            main_time = main_relay_info.get('Time_out')
            backup_time = backup_relay_info.get('Time_out')
            
            # Validar que los tiempos son números
            if not isinstance(main_time, (int, float)) or not isinstance(backup_time, (int, float)):
                skipped_pairs_count += 1
                continue
                
            # Cálculos
            delta_t = backup_time - main_time - CTI
            mt = (delta_t - abs(delta_t)) / 2  # Penalización solo si delta_t es negativo
            
            # Actualizar resultados del escenario
            scenario_results[scenario_id]['tmt'] += mt
            scenario_results[scenario_id]['total_valid'] += 1
            
            if delta_t >= 0:
                scenario_results[scenario_id]['coordinated'] += 1
            else:
                scenario_results[scenario_id]['uncoordinated'] += 1
        
        # 3. Preparar datos para graficar
        data_for_chart = []
        for scenario, data in scenario_results.items():
            data_for_chart.append({
                'Escenario': scenario,
                'TMT': data['tmt'],
                'Pares Coordinados': data['coordinated'],
                'Pares Descoordinados': data['uncoordinated'],
                'Total Pares Válidos': data['total_valid'],
                'Porcentaje Coordinación': 100 * data['coordinated'] / data['total_valid'] if data['total_valid'] > 0 else 0
            })
        
        # Convertir a DataFrame y ordenar por nombre de escenario
        df = pd.DataFrame(data_for_chart)
        df = df.sort_values(by='Escenario')
        
        # 4. Crear gráfico de barras con Plotly
        fig = px.bar(
            df, 
            x='Escenario', 
            y='TMT',
            text='TMT',
            color='TMT',
            color_continuous_scale='Viridis',
            title='Penalización Total por Descoordinación (TMT) por Escenario',
            labels={'TMT': 'Total Miscoordination Time'},
            hover_data=['Pares Coordinados', 'Pares Descoordinados', 'Total Pares Válidos', 'Porcentaje Coordinación']
        )
        
        # Personalizar el gráfico
        fig.update_traces(
            texttemplate='%{text:.5f}',
            textposition='outside'
        )
        
        fig.update_layout(
            xaxis_title='Escenarios',
            yaxis_title='TMT (Total Miscoordination Time)',
            xaxis={'categoryorder': 'array', 'categoryarray': df['Escenario'].tolist()},
            font=dict(size=12),
            height=600,
            width=1000
        )
        
        # 5. Crear un segundo gráfico: porcentaje de coordinación
        fig2 = px.bar(
            df,
            x='Escenario',
            y='Porcentaje Coordinación',
            text='Porcentaje Coordinación',
            color='Porcentaje Coordinación',
            color_continuous_scale='Turbo',
            title='Porcentaje de Pares Coordinados por Escenario',
            labels={'Porcentaje Coordinación': 'Porcentaje de Coordinación (%)'},
            hover_data=['Pares Coordinados', 'Pares Descoordinados', 'Total Pares Válidos', 'TMT']
        )
        
        fig2.update_traces(
            texttemplate='%{text:.1f}%',
            textposition='outside'
        )
        
        fig2.update_layout(
            xaxis_title='Escenarios',
            yaxis_title='Porcentaje de Coordinación (%)',
            xaxis={'categoryorder': 'array', 'categoryarray': df['Escenario'].tolist()},
            font=dict(size=12),
            height=600,
            width=1000
        )
        
        # 5. Mostrar tabla de resultados
        print("\n--- Resultados por Escenario ---")
        print(f"{'Escenario':<15} {'Pares Válidos':<15} {'Coordinados':<15} {'Descoordinados':<15} {'% Coordin.':<15} {'TMT':<15}")
        print("-" * 90)
        
        for index, row in df.iterrows():
            print(f"{row['Escenario']:<15} {int(row['Total Pares Válidos']):<15} {int(row['Pares Coordinados']):<15} {int(row['Pares Descoordinados']):<15} {row['Porcentaje Coordinación']:.1f}%{' ':<10} {row['TMT']:.5f}")
        
        print(f"\nTotal de pares leídos: {total_pairs_read}")
        print(f"Pares omitidos (datos inválidos/faltantes): {skipped_pairs_count}")
        
        # Guardar los gráficos antes de mostrarlos
        output_dir = os.path.dirname(input_file)
        
        # Guardar el primer gráfico (TMT)
        output_file_tmt = os.path.join(output_dir, "tmt_by_scenario.html")
        fig.write_html(output_file_tmt)
        print(f"\nGráfico de TMT guardado en: {output_file_tmt}")
        
        # Guardar el segundo gráfico (% Coordinación)
        output_file_coord = os.path.join(output_dir, "coordination_percentage_by_scenario.html")
        fig2.write_html(output_file_coord)
        print(f"Gráfico de Porcentaje de Coordinación guardado en: {output_file_coord}")
        
        # Mostrar los gráficos
        fig.show()
        fig2.show()
        
    except FileNotFoundError:
        print(f"Error CRÍTICO: No se pudo encontrar el archivo de entrada: {input_file}")
    except TypeError as e:
        print(f"Error CRÍTICO: Problema con el tipo de datos esperado en el archivo JSON: {e}")
    except json.JSONDecodeError as e:
        print(f"Error CRÍTICO: El archivo de entrada JSON ({input_file}) está mal formado: {e}")
    except Exception as e:
        import traceback
        print(f"Error inesperado durante el procesamiento: {e}")
        print("--- Traceback ---")
        traceback.print_exc()
        print("-----------------")
        
if __name__ == "__main__":
    analyze_all_scenarios()


Cargando datos desde: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_second_optimization.json
Calculando TMT por escenario...

--- Resultados por Escenario ---
Escenario       Pares Válidos   Coordinados     Descoordinados  % Coordin.      TMT            
------------------------------------------------------------------------------------------
scenario_1      100             67              33              67.0%           -4.16000
scenario_10     100             90              10              90.0%           -4.03170
scenario_11     100             89              11              89.0%           -4.05650
scenario_12     100             87              13              87.0%           -3.99090
scenario_13     100             91              9               91.0%           -4.08100
scenario_14     100             91              9               91.0%           -4.03190
scenario_15     100             90              10             